In [1]:
from geodata.db.client import WorldDataDB
from geodata.db.models.state import State
from geodata.db.models.city import City

db = WorldDataDB()
COUNTRY_CODE = "DE"

In [2]:
states = []
for state_doc in db.states.coll.find({"country_code": COUNTRY_CODE}):
    state = State(**state_doc)
    if len(state.postal_codes_wikipedia) != 0:
        states.append(state)
len(states)

1

In [3]:
cities = []
for city_doc in db.cities.coll.find({"country_code": COUNTRY_CODE}):
    city = City(**city_doc)
    if len(city.postal_codes_wikipedia) != 0:
        cities.append(city)
len(cities)

6218

In [ ]:

#from geodata.wikipedia.process_postals.common import find_nums_exactly_digits
import re

def find_nums_exactly_digits(text: str, n_digits: int) -> list:
    return re.findall(r'\b(?<!\d)(\d{%d})(?:\s|,|;|\b)(?!\d)\b' % n_digits, text)


def find_num_ranges(text: str) -> list:
    return re.findall(r'\b\d+-\d+\b', text)

# Ejemplo de uso:
texto_ejemplo = "1234; 5678 999das; asd2623, 9873, 4321-6789"
n_digitos = 4
nums_exactamente_digitos = find_nums_exactly_digits(texto_ejemplo, n_digitos)
print(nums_exactamente_digitos)
num_ranges = find_num_ranges(texto_ejemplo)
print(num_ranges)


In [ ]:
rare_cases = {"rare_cases": []}
for city in cities:
    postal = city.postal_codes_wikipedia[0]
    try:
        postal_clean = process_postal_code_de(postal)
    except Exception as e:
        categorize = categorize_postal_code_de(postal)
        print(f"categorize: {categorize}")
        print(postal)
        print(e)
        print("-"*40)
    if postal_clean is None:
        print(f"postal={city.postal_codes_wikipedia} | postal_clean={postal_clean}")
        rare_cases["rare_cases"].append({
            "name_native": city.name,
            "id_csc": city.id_csc,
            "id_wikidata": city.id_wikidata,
            "rare_postal_wikipedia": postal
        })
with open("rare_cases_DE2.json", "w") as f:
    json.dump(rare_cases, f)

In [ ]:
from geodata.wikipedia.process_postals.de import process_postal_code_de


postal_codes = [
    "(nr. 0310) 13127",
    "(nr. 0702) 12159, 12161",
    #"1234",
    #"1112345-1112355",
    #"12345, 67890, 123456",
    #"9827-9829, 12345, 67890-67895",
    #"78142-78144, 78701-78713",
    #"invalid_code"
]

for code in postal_codes:
    print(f"Postal code: {code}")
    print(f"Processed: {process_postal_code_de(code)}")
    print()


In [ ]:
rare_cases = {"rare_cases": []}
for city in cities:
    postal = city.postal_codes_wikipedia[0]
    if categorize_postal_code(postal) == "INVALID":
        rare_cases["rare_cases"].append({
            "name_native": city.name_native,
            "id_csc": city.id_csc,
            "id_wikidata": city.id_wikidata,
            "rare_postal_wikipedia": postal
        })

In [ ]:
import json

with open("rare_cases_DE.json", "w") as f:
    json.dump(rare_cases, f)

In [ ]:


text = is_only_numbers("")
text

In [ ]:
cities_2 = []
for city in cities:
    for postal in city.postal_codes_wikipedia:
        for char in postal:
            if char not in "0123456789-, ":
                cities_2.append(city)
                break
cities_2